# Boostrapping Curva SOFR

## Configuración

### Librerías

In [1]:
from scipy.optimize import newton
from datetime import date
import pandas as pd
import numpy as np

from finrisk import QC_Financial_3 as Qcf
import my_functions as myf

### Fecha de Cálculo

In [2]:
fecha_val = Qcf.QCDate(15, 10, 2021)

## Input para la Curva

In [3]:
archivo = 'data/data_curva_sofr.xlsm'

In [4]:
data = pd.read_excel(archivo, dtypes={'start_date': date, 'end_date': date, 'nominal': float})

In [5]:
data.head(10).style.format({'rate': '{:.5%}'})

,bbg ticker,rate,start_date,end_date,tipo,tenor,periodicity,stub_period,settlement_lag,rolling,nominal,amort_is_cashflow,currency,interest_rate
0,SOFRRATE Index,0.05000%,2021-10-15 00:00:00,2021-10-19 00:00:00,TASA,2D,1Y,SHORT_FRONT,0,MOD_FOLLOW,1000000,True,USD,LINACT360
1,USOSFR1Z BGN Curncy,0.05210%,2021-10-19 00:00:00,2021-10-28 00:00:00,TASA,7D,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
2,USOSFR2Z BGN Curncy,0.05240%,2021-10-19 00:00:00,2021-11-04 00:00:00,TASA,14D,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
3,USOSFR3Z BGN Curncy,0.05220%,2021-10-19 00:00:00,2021-11-11 00:00:00,TASA,21D,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
4,USOSFRA BGN Curncy,0.05200%,2021-10-19 00:00:00,2021-11-19 00:00:00,SWAP,1M,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
5,USOSFRB BGN Curncy,0.05260%,2021-10-19 00:00:00,2021-12-19 00:00:00,SWAP,2M,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
6,USOSFRC BGN Curncy,0.05400%,2021-10-19 00:00:00,2022-01-19 00:00:00,SWAP,3M,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
7,USOSFRD BGN Curncy,0.05610%,2021-10-19 00:00:00,2022-02-19 00:00:00,SWAP,4M,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
8,USOSFRE BGN Curncy,0.05570%,2021-10-19 00:00:00,2022-03-19 00:00:00,SWAP,5M,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
9,USOSFRF BGN Curncy,0.06020%,2021-10-19 00:00:00,2022-04-19 00:00:00,SWAP,6M,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360


## Construcción de Operaciones

### Patas Fijas

In [6]:
act360 = Qcf.QCAct360()
lin_wf = Qcf.QCLinearWf()
cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2021), 50)
clp = Qcf.QCCLP()
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodo_irregular = Qcf.StubPeriod.SHORTFRONT
es_bono = False

In [7]:
patas_fijas = []
for row in data.itertuples():
    if row.tipo == 'TASA':
        fecha_inicio = Qcf.build_qcdate_from_string(
            row.start_date.date().isoformat())
        
        fecha_final = Qcf.build_qcdate_from_string(
            row.end_date.date().isoformat())
        
        fecha_pago = cal.shift(fecha_final, row.settlement_lag)
        
        tasa = Qcf.QCInterestRate(row.rate, act360, lin_wf)
        
        fixed_rate_cashflow = Qcf.FixedRateCashflow(
            fecha_inicio,
            fecha_final,
            fecha_pago,
            row.nominal,
            row.nominal,
            row.amort_is_cashflow,
            tasa,
            clp
        )
        leg = Qcf.Leg()
        leg.append_cashflow(fixed_rate_cashflow)
        patas_fijas.append(leg)
    else:
        fecha_inicio = Qcf.build_qcdate_from_string(row.start_date.date().isoformat())
        
        fecha_final = Qcf.build_qcdate_from_string(row.end_date.date().isoformat())
        
        fixed_rate_leg = Qcf.LegFactory.build_bullet_fixed_rate_leg(
            Qcf.RecPay.RECEIVE,
            fecha_inicio,
            fecha_final,
            bus_adj_rule,
            Qcf.Tenor(row.periodicity),
            periodo_irregular,
            cal,
            row.settlement_lag,
            row.nominal,
            row.amort_is_cashflow,
            Qcf.QCInterestRate(row.rate, act360, lin_wf),
            clp,
            es_bono
        )
        patas_fijas.append(fixed_rate_leg)

In [12]:
myf.leg_as_dataframe(patas_fijas[22], myf.TipoPata.FIJA)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2021-10-19,2022-10-19,2022-10-21,1000000.0,0.0,11656.680556,True,1.165668e+04,CLP,0.011497,LinAct360
1,2022-10-19,2023-10-19,2023-10-23,1000000.0,0.0,11656.680556,True,1.165668e+04,CLP,0.011497,LinAct360
2,2023-10-19,2024-10-21,2024-10-23,1000000.0,0.0,11752.488889,True,1.175249e+04,CLP,0.011497,LinAct360
3,2024-10-21,2025-10-20,2025-10-22,1000000.0,0.0,11624.744444,True,1.162474e+04,CLP,0.011497,LinAct360
4,2025-10-20,2026-10-19,2026-10-21,1000000.0,0.0,11624.744444,True,1.162474e+04,CLP,0.011497,LinAct360
5,2026-10-19,2027-10-19,2027-10-21,1000000.0,0.0,11656.680556,True,1.165668e+04,CLP,0.011497,LinAct360
6,2027-10-19,2028-10-19,2028-10-23,1000000.0,1000000.0,11688.616667,True,1.011689e+06,CLP,0.011497,LinAct360


### Patas OIS

Vamos a usar el tipo de cashflow `IcpClpCashflow2`.

In [13]:
patas_ois = []
start_date_index = 1.0
end_date_index = 1.0
for row in data.itertuples():
    if row.tipo == 'TASA':
        fecha_inicio = Qcf.build_qcdate_from_string(
            row.start_date.date().isoformat())
        
        fecha_final = Qcf.build_qcdate_from_string(
            row.end_date.date().isoformat())
        
        fecha_pago = cal.shift(fecha_final, row.settlement_lag)
        
        icp_clp_cashflow = Qcf.IcpClpCashflow2(
            fecha_inicio,
            fecha_final,
            fecha_pago,
            -row.nominal,
            -row.nominal,
            row.amort_is_cashflow,
            0.0, # spread
            1.0, # gearing
            start_date_index,
            end_date_index
        )
        leg = Qcf.Leg()
        leg.append_cashflow(icp_clp_cashflow)
        patas_ois.append(leg)
    else:
        fecha_inicio = Qcf.build_qcdate_from_string(
            row.start_date.date().isoformat())
        
        fecha_final = Qcf.build_qcdate_from_string(
            row.end_date.date().isoformat())
        
        icp_clp_leg = Qcf.LegFactory.build_bullet_icp_clp2_leg(
            Qcf.RecPay.PAY,
            fecha_inicio,
            fecha_final,
            bus_adj_rule,
            Qcf.Tenor(row.periodicity),
            periodo_irregular,
            cal,
            row.settlement_lag, 
            row.nominal,
            row.amort_is_cashflow,
            0.0, # spread
            1.0, # gearing
            True # La tasa es LinAct/360 con False la tasa es Lin30/360
        )
        patas_ois.append(icp_clp_leg)

In [14]:
myf.leg_as_dataframe(patas_ois[22], myf.TipoPata.ICPCLP)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2021-10-19,2022-10-19,2022-10-21,-1000000.0,0.0,True,0.0,CLP,10000.0,10000.0,0.0,-0.0,0.0,1.0,LinAct360
1,2022-10-19,2023-10-19,2023-10-23,-1000000.0,0.0,True,0.0,CLP,10000.0,10000.0,0.0,-0.0,0.0,1.0,LinAct360
2,2023-10-19,2024-10-21,2024-10-23,-1000000.0,0.0,True,0.0,CLP,10000.0,10000.0,0.0,-0.0,0.0,1.0,LinAct360
3,2024-10-21,2025-10-20,2025-10-22,-1000000.0,0.0,True,0.0,CLP,10000.0,10000.0,0.0,-0.0,0.0,1.0,LinAct360
4,2025-10-20,2026-10-19,2026-10-21,-1000000.0,0.0,True,0.0,CLP,10000.0,10000.0,0.0,-0.0,0.0,1.0,LinAct360
5,2026-10-19,2027-10-19,2027-10-21,-1000000.0,0.0,True,0.0,CLP,10000.0,10000.0,0.0,-0.0,0.0,1.0,LinAct360
6,2027-10-19,2028-10-19,2028-10-23,-1000000.0,-1000000.0,True,-1000000.0,CLP,10000.0,10000.0,0.0,-0.0,0.0,1.0,LinAct360


## Función Objetivo

In [15]:
act365 = Qcf.QCAct365()
exp_wf = Qcf.QCContinousWf()

In [31]:
plazos = Qcf.long_vec()
for p in patas_fijas:
    num = p.size()
    fecha = p.get_cashflow_at(num - 1).get_settlement_date()
    plazos.append(fecha_val.day_diff(fecha))
    print(fecha_val.day_diff(fecha))

4
17
24
31
39
68
98
131
159
188
220
250
279
312
341
371
553
738
1104
1468
1832
2197
2565
2930
3295
3659
4389
5486
7313
9139
10965
14616
18268


In [17]:
tasas_ini = [0.0 for i in patas_fijas]

In [18]:
vp = Qcf.PresentValue()
fwd = Qcf.ForwardRates()

In [21]:
def obj(tasa, tasas, cual):
    # Construir un objeto curva cero cupón de Qcf
    ctasas = Qcf.double_vec()
    for r in tasas:
        ctasas.append(r)
    zcc = Qcf.QCCurve(plazos, ctasas)
    plazo = zcc.get_values_at(cual).tenor
    zcc.set_pair(plazo, float(tasa))
    lin = Qcf.QCLinearInterpolator(zcc)
    zz = Qcf.ZeroCouponCurve(lin, Qcf.QCInterestRate(0.0, act365, exp_wf))
    
    # Calculo el valor de la pata fija del swap de índice cual
    vp_fix = vp.pv(fecha_val, patas_fijas[cual], zz)
    # print(vp_fix)
    
    # Calculo las proyecciones de los índices de la pata OIS correspondiente
    fwd.set_rates_icp_clp_leg(fecha_val, 1.0, patas_ois[cual], zz)
    
    # Calculo el valor presente de la pata OIS
    vp_flot = vp.pv(fecha_val, patas_ois[cual], zz)
    # print(vp_flot)
    
    return vp_fix + vp_flot

In [27]:
obj(0.0005069, tasas_ini, 0)

0.0004716303665190935

## Solución

In [29]:
for cual in range(len(patas_fijas)):
    res = newton(obj, 0.0, args=(tasas_ini, cual), tol=1.e-10, disp=True)
    tasas_ini[cual] = res

In [30]:
for r in tasas_ini:
    print(f'{r:.5%}')

0.05069%
0.05282%
0.05240%
0.05286%
0.05212%
0.05325%
0.05467%
0.05681%
0.05628%
0.06114%
0.06397%
0.07078%
0.07977%
0.09276%
0.10284%
0.11696%
0.23675%
0.38658%
0.64294%
0.82872%
0.96517%
1.07815%
1.17015%
1.24262%
1.30101%
1.35263%
1.43250%
1.51157%
1.58022%
1.58920%
1.57695%
1.48615%
1.36520%
